In [1]:
import seaborn as sb
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Grouper
from pandas import DataFrame
import csv
import pickle
import math
import calendar 

/home/jonathan/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv("/home/jonathan/tesis/Data_Temperatura_Lluvia/2020_Accidentalidad.csv",sep=';')#,index_col=['fecha'], parse_dates=True)
DataAcc = df.drop(columns=['localizacion','numero','estado_meteorológico','tipo_vehiculo','tipo_persona','rango_edad','sexo','cod_lesividad','lesividad','coordenada_x_utm','coordenada_y_utm','positiva_alcohol','positiva_droga'])
DataAcc['ds'] = DataAcc['fecha'] + '/' + DataAcc['hora']
DataAcc['ds'] = pd.to_datetime(DataAcc['ds'], format='%d/%m/%Y/%H:%M:%S')
DataAcc = DataAcc.drop(columns=['fecha','hora'])
DataAcc = DataAcc.set_index('ds')
DataAcc = DataAcc.sort_index()
DataAcc

,num_expediente,cod_distrito,distrito,tipo_accidente
ds,,,,
2020-01-01 01:00:00,2020S000014,3.0,RETIRO,Colisión fronto-lateral
2020-01-01 01:00:00,2020S000004,15.0,CIUDAD LINEAL,Choque contra obstáculo fijo
2020-01-01 01:00:00,2020S000004,15.0,CIUDAD LINEAL,Choque contra obstáculo fijo
2020-01-01 01:00:00,2020S000004,15.0,CIUDAD LINEAL,Choque contra obstáculo fijo
2020-01-01 01:00:00,2020S000014,3.0,RETIRO,Colisión fronto-lateral
...,...,...,...,...
2020-12-31 21:55:00,2020S019513,19.0,VICÁLVARO,Atropello a persona
2020-12-31 22:00:00,2020S019516,12.0,USERA,Choque contra obstáculo fijo
2020-12-31 22:25:00,2020S019517,13.0,PUENTE DE VALLECAS,Choque contra obstáculo fijo


In [3]:
DataAcc = DataAcc.loc[((DataAcc['cod_distrito'] == 16) | (DataAcc['cod_distrito'] == 8) | (DataAcc['cod_distrito'] == 5) | (DataAcc['cod_distrito'] == 7))# | (DataAcc['cod_distrito'] == 7) | (DataAcc['cod_distrito'] == 1) | (DataAcc['cod_distrito'] == 4))]  
                       & ((DataAcc['tipo_accidente'] != 'Caída') & (DataAcc['tipo_accidente'] != 'Otro') #& (DataAcc['tipo_accidente'] != 'Vuelco')
                        & (DataAcc['tipo_accidente'] != 'Atropello a animal') & (DataAcc['tipo_accidente'] != 'Colisión frontal')
                         & (DataAcc['tipo_accidente'] != 'Solo salida de la vía'))]
DataAcc['Accidente'] = True
DataAcc = DataAcc.drop_duplicates(subset = ['num_expediente'])

/tmp/ipykernel_10695/4244713018.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DataAcc['Accidente'] = True


In [4]:
# # Observar hora exacta de accidente
data1=DataAcc.loc['2020-07-06':'2020-07-06']
data1.head(60)

,num_expediente,cod_distrito,distrito,tipo_accidente,Accidente
ds,,,,,
2020-07-06 10:45:00,2020S009407,7.0,CHAMBERÍ,Atropello a persona,True
2020-07-06 12:00:00,2020S009461,5.0,CHAMARTÍN,Choque contra obstáculo fijo,True
2020-07-06 12:08:00,2020S009507,16.0,HORTALEZA,Colisión fronto-lateral,True
2020-07-06 17:05:00,2020S009427,8.0,FUENCARRAL-EL PARDO,Choque contra obstáculo fijo,True
2020-07-06 19:22:00,2020S009472,5.0,CHAMARTÍN,Choque contra obstáculo fijo,True
2020-07-06 21:10:00,2020S009536,8.0,FUENCARRAL-EL PARDO,Atropello a persona,True


In [5]:
DataAcc = DataAcc.reset_index()

In [6]:
#Patrón Pardo: Hora de accidente es una hora antes y afEcta la hora siguiente si y solo si el minuto de la hora resultante es >=10
#Patrón Hortaleza: Accidente tiene efecto una hora después si y solo si el minuto de la hora es <10, accidente por alcance no tienen ningún efecto y accidente en la noche no tiene efecto (23:00)
#Patrón Chamartín: La hora de accidente real es la hora registrada pero afecta a la hora siguiente, accidente en la madrugada no tiene efecto 
def ModTime(ds):
    global valor
    global fila  
    global prueba
    date = pd.to_datetime(ds)
    hora = date.hour
    minuto = date.minute    
    dia = date.day
    mes = date.month    
    consulta = DataAcc.loc[fila]    
    distrito = consulta['distrito']        
    
    if (distrito == 'FUENCARRAL-EL PARDO'):
#         try:
#             if (hora > 0) & (minuto == 0):
#                 date = date.replace(hour=hora-1,minute=0)
#             elif (hora == 0) & (minuto == 0):
#                 date = date.replace(day=dia-1,hour=23)
#             elif (hora > 0) & (minuto < 10): ############REVISAR
#                 date = date.replace(hour=hora-1,minute=0)
#             elif (hora == 0) & (minuto < 10):
#                 date = date.replace(day=dia-1,hour=23,minute=0)                
#             else:
#                 date = date.replace(hour=hora,minute=0)
#         except:
#             res = calendar.monthrange(date.year, date.month)
#             lday = res[1]
#             date = date.replace(month=mes-1, day=lday, hour=23, minute=0)  

        try:
            if (hora < 23) & (minuto >= 0):
                date = date.replace(hour=hora+1,minute=0)                        
            elif (hora == 23) & (minuto >= 0):
                date = date.replace(day=dia+1,hour=0,minute=0)                            
        except:            
            date = date.replace(month=mes+1, day=1, hour=0, minute=0)  
            
    elif (distrito == 'HORTALEZA'):
        try:
            if (hora < 23) & (minuto == 0):
                date = date.replace(hour=hora+1,minute=0)
            elif (hora == 23) & (minuto == 0):
                date = date.replace(day=dia+1,hour=0)
            elif (hora < 23) & (minuto <= 45): ############REVISAR
                date = date.replace(hour=hora+1,minute=0)
            elif (hora == 23) & (minuto <= 45):
                date = date.replace(day=dia+1,hour=0,minute=0)    
            elif (hora < 22) & (minuto > 45):        
                date = date.replace(hour=hora+2,minute=0)         
            elif (hora == 22) & (minuto > 45):
                date = date.replace(day=dia+1,hour=0,minute=0)
            elif (hora == 23) & (minuto > 45):
                date = date.replace(day=dia+1,hour=1,minute=0)
        except:               
            if (hora == 23) & (minuto == 0):
                date = date.replace(month=mes+1,day=1,hour=0)            
            elif (hora == 23) & (minuto <= 45):
                date = date.replace(month=mes+1,day=1,hour=0,minute=0)                                
            elif (hora == 22) & (minuto > 45):
                date = date.replace(month=mes+1,day=1,hour=0,minute=0)
            elif (hora == 23) & (minuto > 45):
                date = date.replace(month=mes+1,day=1,hour=1,minute=0)              
                
    elif (distrito == 'CHAMARTÍN'):           
#         try:
#             if (hora < 23) & (minuto == 0):                
#                 date = date.replace(hour=hora+1,minute=0)
#             elif (hora == 23) & (minuto == 0):                
#                 date = date.replace(day=dia+1,hour=0)
#             elif (hora < 23) & (minuto < 45):               
#                 date = date.replace(hour=hora+1,minute=0)            
#             elif (hora == 23) & (minuto < 45):                
#                 date = date.replace(day=dia+1,hour=0,minute=0)                
#             elif (hora < 22) & (minuto >= 45):                
#                 date = date.replace(hour=hora+2,minute=0)         
#             elif (hora == 22) & (minuto >= 45):                
#                 date = date.replace(day=dia+1,hour=0,minute=0)
#             elif (hora == 23) & (minuto >= 45):                
#                 date = date.replace(day=dia+1,hour=1,minute=0)            
#         except:            
#             if (hora == 23) & (minuto == 0):                
#                 date = date.replace(month=mes+1,day=1,hour=0)            
#             elif (hora == 23) & (minuto < 45):                
#                 date = date.replace(month=mes+1,day=1,hour=0,minute=0)                                
#             elif (hora == 22) & (minuto >= 45):                
#                 date = date.replace(month=mes+1,day=1,hour=0,minute=0)
#             elif (hora == 23) & (minuto >= 45):                
#                 date = date.replace(month=mes+1,day=1,hour=1,minute=0)    
        try:
            if (hora < 23) & (minuto == 0):                
                date = date.replace(hour=hora+1,minute=0)
            elif (hora == 23) & (minuto == 0):                
                date = date.replace(day=dia+1,hour=0)
            elif (hora < 22) & (minuto != 0):               
                date = date.replace(hour=hora+2,minute=0)            
            elif (hora == 22) & (minuto != 0):                
                date = date.replace(day=dia+1,hour=0,minute=0)                
            elif (hora == 23) & (minuto != 0):                
                date = date.replace(day=dia+1,hour=1,minute=0)                            
        except:            
            if (hora == 23) & (minuto == 0):                
                date = date.replace(month=mes+1,day=1,hour=0)                        
            elif (hora == 22) & (minuto != 0):                
                date = date.replace(month=mes+1,day=1,hour=0,minute=0)
            elif (hora == 23) & (minuto != 0):                
                date = date.replace(month=mes+1,day=1,hour=1,minute=0)    
                
    elif (distrito == 'CHAMBERÍ'):           
        try:
            if (minuto == 0):                
                date = date.replace(hour=hora,minute=0)            
            elif (minuto <= 10 ):               
                date = date.replace(hour=hora,minute=0)                       
            elif (hora < 23) & (minuto > 10):                
                date = date.replace(hour=hora+1,minute=0)                     
            elif (hora == 23) & (minuto > 10):                
                date = date.replace(day=dia+1,hour=0,minute=0)            
        except:            
            if (hora == 23):                
                date = date.replace(month=mes+1,day=1,hour=0)                        
    fila += 1
    
    return (date)
prueba=0
fila = 0
DataAcc['ds1'] = DataAcc['ds'].apply(ModTime)
# DataAcc = DataAcc.drop_duplicates(subset = ['ds']) # Contabilizar en una nueva columna las repeticiones del index

In [7]:
def ds2(ds):
    date = pd.to_datetime(ds)
    año = date.year
    mes = date.month
    dia = date.day
    hora = date.hour    
    try:
        if (hora < 23):   # Revisar si esta bien esta condición
            date = date.replace(hour=hora+1)        
        elif (hora == 23):
            date = date.replace(day=dia+1,hour=0)
    except:        
        if (mes < 12):
            date = date.replace(month=mes+1, day=1, hour=0)         
        else:
            date = date.replace(year=año+1, month=1, day=1, hour=0)

#     try:
#         if (hora < 22):   # Revisar si esta bien esta condición
#             date = date.replace(hour=hora+2)        
#         elif (hora == 22):
#             date = date.replace(day=dia+1,hour=0)
#         elif (hora == 23):
#             date = date.replace(day=dia+1,hour=1)
#     except:        
#         if (mes < 12) & (hora == 22):
#             date = date.replace(month=mes+1, day=1, hour=0)         
#         elif (mes < 12) & (hora == 23):
#             date = date.replace(month=mes+1, day=1, hour=1)         
#         elif (mes == 12) & (hora == 22):
#             date = date.replace(year=año+1, month=1, day=1, hour=0)
#         elif (mes == 12) & (hora == 23):
#             date = date.replace(year=año+1, month=1, day=1, hour=1)
            
    return (date)

DataAcc['ds2'] = DataAcc['ds1'].apply(ds2)

In [8]:
DataAcc = DataAcc.set_index('ds')

In [9]:
Prueba = DataAcc.loc['2020-07-06':'2020-07-06']
Prueba

,num_expediente,cod_distrito,distrito,tipo_accidente,Accidente,ds1,ds2
ds,,,,,,,
2020-07-06 10:45:00,2020S009407,7.0,CHAMBERÍ,Atropello a persona,True,2020-07-06 11:00:00,2020-07-06 12:00:00
2020-07-06 12:00:00,2020S009461,5.0,CHAMARTÍN,Choque contra obstáculo fijo,True,2020-07-06 13:00:00,2020-07-06 14:00:00
2020-07-06 12:08:00,2020S009507,16.0,HORTALEZA,Colisión fronto-lateral,True,2020-07-06 13:00:00,2020-07-06 14:00:00
2020-07-06 17:05:00,2020S009427,8.0,FUENCARRAL-EL PARDO,Choque contra obstáculo fijo,True,2020-07-06 18:00:00,2020-07-06 19:00:00
2020-07-06 19:22:00,2020S009472,5.0,CHAMARTÍN,Choque contra obstáculo fijo,True,2020-07-06 21:00:00,2020-07-06 22:00:00
2020-07-06 21:10:00,2020S009536,8.0,FUENCARRAL-EL PARDO,Atropello a persona,True,2020-07-06 22:00:00,2020-07-06 23:00:00


In [10]:
Prueba = DataAcc.loc['2020-07-13':'2020-07-13']
Prueba

,num_expediente,cod_distrito,distrito,tipo_accidente,Accidente,ds1,ds2
ds,,,,,,,
2020-07-13 13:45:00,2020S009778,7.0,CHAMBERÍ,Colisión fronto-lateral,True,2020-07-13 14:00:00,2020-07-13 15:00:00
2020-07-13 14:35:00,2020S009752,7.0,CHAMBERÍ,Alcance,True,2020-07-13 15:00:00,2020-07-13 16:00:00
2020-07-13 15:10:00,2020S009801,5.0,CHAMARTÍN,Alcance,True,2020-07-13 17:00:00,2020-07-13 18:00:00
2020-07-13 15:40:00,2020S009797,5.0,CHAMARTÍN,Colisión lateral,True,2020-07-13 17:00:00,2020-07-13 18:00:00
2020-07-13 19:40:00,2020S009798,8.0,FUENCARRAL-EL PARDO,Colisión fronto-lateral,True,2020-07-13 20:00:00,2020-07-13 21:00:00
2020-07-13 20:18:00,2020S009764,7.0,CHAMBERÍ,Colisión lateral,True,2020-07-13 21:00:00,2020-07-13 22:00:00


In [11]:
Prueba = DataAcc.loc['2020-07-20':'2020-07-20']
Prueba

,num_expediente,cod_distrito,distrito,tipo_accidente,Accidente,ds1,ds2
ds,,,,,,,
2020-07-20 06:00:00,2020S010098,16.0,HORTALEZA,Choque contra obstáculo fijo,True,2020-07-20 07:00:00,2020-07-20 08:00:00
2020-07-20 09:15:00,2020S010113,7.0,CHAMBERÍ,Alcance,True,2020-07-20 10:00:00,2020-07-20 11:00:00
2020-07-20 12:10:00,2020S010159,5.0,CHAMARTÍN,Colisión fronto-lateral,True,2020-07-20 14:00:00,2020-07-20 15:00:00
2020-07-20 12:55:00,2020S010130,16.0,HORTALEZA,Choque contra obstáculo fijo,True,2020-07-20 14:00:00,2020-07-20 15:00:00
2020-07-20 13:30:00,2020S010155,7.0,CHAMBERÍ,Alcance,True,2020-07-20 14:00:00,2020-07-20 15:00:00
2020-07-20 14:48:00,2020S010122,7.0,CHAMBERÍ,Colisión fronto-lateral,True,2020-07-20 15:00:00,2020-07-20 16:00:00
2020-07-20 17:05:00,2020S010174,5.0,CHAMARTÍN,Colisión fronto-lateral,True,2020-07-20 19:00:00,2020-07-20 20:00:00
2020-07-20 18:45:00,2020S010175,5.0,CHAMARTÍN,Alcance,True,2020-07-20 20:00:00,2020-07-20 21:00:00
2020-07-20 19:30:00,2020S010185,8.0,FUENCARRAL-EL PARDO,Colisión fronto-lateral,True,2020-07-20 20:00:00,2020-07-20 21:00:00


In [12]:
Prueba = DataAcc.loc['2020-07-27':'2020-07-27']
Prueba

,num_expediente,cod_distrito,distrito,tipo_accidente,Accidente,ds1,ds2
ds,,,,,,,
2020-07-27 02:45:00,2020S010458,5.0,CHAMARTÍN,Choque contra obstáculo fijo,True,2020-07-27 04:00:00,2020-07-27 05:00:00
2020-07-27 09:10:00,2020S011678,5.0,CHAMARTÍN,Choque contra obstáculo fijo,True,2020-07-27 11:00:00,2020-07-27 12:00:00
2020-07-27 09:25:00,2020S010466,7.0,CHAMBERÍ,Colisión fronto-lateral,True,2020-07-27 10:00:00,2020-07-27 11:00:00
2020-07-27 13:10:00,2020S010512,7.0,CHAMBERÍ,Alcance,True,2020-07-27 13:00:00,2020-07-27 14:00:00
2020-07-27 15:30:00,2020S010477,7.0,CHAMBERÍ,Colisión fronto-lateral,True,2020-07-27 16:00:00,2020-07-27 17:00:00
2020-07-27 20:30:00,2020S010542,5.0,CHAMARTÍN,Alcance,True,2020-07-27 22:00:00,2020-07-27 23:00:00
2020-07-27 21:40:00,2020S010536,8.0,FUENCARRAL-EL PARDO,Alcance,True,2020-07-27 22:00:00,2020-07-27 23:00:00
2020-07-27 21:45:00,2020S010544,16.0,HORTALEZA,Alcance,True,2020-07-27 22:00:00,2020-07-27 23:00:00
2020-07-27 21:45:00,2020S010538,16.0,HORTALEZA,Colisión fronto-lateral,True,2020-07-27 22:00:00,2020-07-27 23:00:00


In [13]:
# Prueba = DataAcc.loc['2020-07-27':'2020-07-27']
# Prueba

In [14]:
# Acc_ds = DataAcc.loc[:,['Accidente']]
# DataAcc = DataAcc.set_index('ds1')
# Acc_ds1 = DataAcc.loc[:,['Accidente']]


# Acc_ds1 = Acc_ds1.reset_index() 
# Acc_ds1 = Acc_ds1.drop_duplicates(subset = ['ds1'])
# Acc_ds1 = Acc_ds1.set_index('ds1') 

In [15]:
# Acc_ds1
# Data=Acc_ds1.loc['2020-07-20':'2020-07-20']
# Data

In [16]:
# Acc_ds.to_csv("Acc2_ds.csv")
# Acc_ds1.to_csv("Acc2_ds1.csv")
# Acc_ds2.to_csv("Acc2_ds2.csv")